# Import librairies

In [240]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.mixture import GaussianMixture
import tensorflow as tf
from sklearn.model_selection import train_test_split


from tensorflow.keras.layers import (
    Input,
    Embedding,
    Concatenate,
    Flatten,
    Dense,
    Dropout,
    concatenate,
)
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD, Adam

# Import data

In [2]:
path = "../data"
df = pd.read_csv(os.path.join(path,"cleaned_dataset.csv"),index_col=0)

In [102]:
df = df[df["basesalary"]<=2*df["basesalary"].mean()]

# Add gaussian noise by columns

In [105]:
noises_std={}
for col in df.columns:
    noises_std[col] = df[col].std()    

In [106]:
df2 = df.copy()

In [107]:
for col in df2.columns:
    df2[col]=df2[col]+np.random.normal(0,noises_std[col],len(df))

In [108]:
X = df2.drop(columns=["basesalary"])
y = df2["basesalary"]

In [109]:
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0, max_depth=10)
regressor.fit(X, y) 
y_pred = regressor.predict(X)
rmse = float(format(np.sqrt(mean_squared_error(y, y_pred)), '.3f'))
print("\nRMSE: ", rmse)


RMSE:  47122.597


In [110]:
y_pred = regressor.predict(df.drop(columns=["basesalary"]))
rmse = float(format(np.sqrt(mean_squared_error(df["basesalary"], y_pred)), '.3f'))
print("\nRMSE: ", rmse)


RMSE:  28458.664


In [111]:
pd.concat([df2["yearsofexperience"],df["yearsofexperience"]],axis=1).corr()

,yearsofexperience,yearsofexperience
yearsofexperience,1.000000,0.708437
yearsofexperience,0.708437,1.000000


# Add gaussian noise by rows

# Gaussian mixture

In [113]:
gm = GaussianMixture(n_components=20, random_state=0,covariance_type='diag').fit(df)

In [114]:
df_sample = gm.sample(50000)[0]
y_gaussian = df_sample[:,3]
x_gaussian = df_sample[:,[i for i in range(len(df.columns)) if i!=3]]
regressor = RandomForestRegressor(n_estimators = 30, random_state = 0, max_depth=10)
regressor.fit(x_gaussian, y_gaussian) 

RandomForestRegressor(max_depth=10, n_estimators=30, random_state=0)

In [115]:
rmse = float(format(np.sqrt(mean_squared_error(y, y_pred)), '.3f'))
print("\nRMSE: ", rmse)


RMSE:  48668.581


In [116]:
y_pred = regressor.predict(df.drop(columns=["basesalary"]))
rmse = float(format(np.sqrt(mean_squared_error(df["basesalary"], y_pred)), '.3f'))
print("\nRMSE: ", rmse)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(



RMSE:  160890.363


In [117]:
df["basesalary"].mean()

155892.7930736961

# Word embedding and then gaussian mixture

In [224]:
path = "../data"
df = pd.read_csv(os.path.join(path,"data_with_missing.csv"),index_col=0)
features_to_keep = ["company","level","title","location","yearsofexperience","yearsatcompany","gender","year","tag","tag_truncated","country"]
label =["basesalary"]
df = df[features_to_keep+label]
df.loc[df["level"].isin(df["level"].value_counts()[df["level"].value_counts()<=50].keys().tolist()),"level"]="other"
df.loc[df["tag"].isin(df["tag"].value_counts()[df["tag"].value_counts()<=50].keys().tolist()),"tag"]="other"

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [225]:
#for column in ["tag","tag_truncated"]:
#   df[column]=df[column].apply(lambda x: df[column].mode().tolist()[0] if pd.isnull(x) else x)
df["year"]=df["year"].apply(lambda x: x if pd.isnull(x) else x[:4])
df = df[df["basesalary"]!=0]
df = df[df["basesalary"]<=2*df["basesalary"].mean()]

In [226]:
ohe_features=["title","location","gender","year","country"]
textual_features=["company","level","tag","tag_truncated"]
numeric_features=["yearsofexperience","yearsatcompany"]
label=["basesalary"]
df=df[ohe_features+textual_features+numeric_features+label]

In [227]:
for feature in textual_features:
    df.loc[pd.isnull(df[feature]),feature]="Unavailable"

In [228]:
vocab = {}

# textual_feature =
for feature in textual_features:
    df[feature]=df[feature]
    vocab[feature] = {
        k: v for v, k in enumerate(np.unique(df[feature].apply(str)))
    }
    # print(vocab[feature])
    df[feature] = df[feature].apply(lambda x: vocab[feature][x])


In [229]:
df = pd.get_dummies(df, columns=ohe_features)

In [237]:
L_ohe=df.drop(columns=numeric_features+label+textual_features).columns

In [195]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=["basesalary"]),df["basesalary"])

In [201]:
dict_shape = {}
for cf in textual_features:
    dict_shape[cf] = len(df[cf].unique())


In [276]:
input_ohe_dict

{'title_Business Analyst': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_title_Business Analyst')>,
 'title_Data Scientist': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_title_Data Scientist')>,
 'title_Hardware Engineer': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_title_Hardware Engineer')>,
 'title_Human Resources': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_title_Human Resources')>,
 'title_Management Consultant': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_title_Management Consultant')>,
 'title_Marketing': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_title_Marketing')>,
 'title_Mechanical Engineer': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_title_Mechanical Engineer')>,
 'title_Product Designer': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_title_Product Designer')>,
 'title_Product Ma

In [287]:
input_textual_dict = {}
input_numeric_dict = {}
input_ohe_dict = {}
for feature in textual_features:
    input_textual_dict[feature] = Input(shape=(1,), name=f"input_{feature}")

for feature in numeric_features:
    input_numeric_dict[feature] = Input(shape=(1,), name=f"input_{feature}")

for feature in L_ohe:
    input_ohe_dict[feature] = Input(shape=(1,), name=f"input_{feature}")
    
inputs = [input_textual_dict,input_numeric_dict,input_ohe_dict]    
k_embedding = {feature: 20 for feature in textual_features}

embeddings = {}
for feature in textual_features:
    embeddings[feature] = Flatten(name=f"Flatten_{feature}")(
        Embedding(
            input_dim=dict_shape[feature],
            output_dim=k_embedding[feature],
            input_length=(1,),
            name=f"Embedding_{feature}",
        )(input_textual_dict[feature])
    )
    
concatted = Concatenate(name="concatted")(
                list(embeddings.values())
                + [elem for inpt in [input_numeric_dict,input_ohe_dict] for elem in inpt.values()],
            )
dense_layer0 = Dense(50, name="Dense0")(concatted)
dense_layer1 = Dense(20, name="Dense1")(dense_layer0)
out = Dense(1, name="out")(dense_layer1)
model = Model(inputs=inputs, outputs=out, name="embedding_model")

In [299]:
model.compile(optimizer=Adam(learning_rate=0.01),
            loss="mse",
            metrics=["mae"],
            # run_eagerly=True,
        )
epochs = 30
#model.fit([X_train[textual_features],X_train[numeric_features],X_train[L_ohe]],y_train,epochs=epochs,
#   batch_size=256)

model.fit(np.array([[df[textual_features].iloc[i],df[numeric_features].iloc[i],df[L_ohe].iloc[i] ]for i in range(10)]),y_train.iloc[:10],epochs=epochs,
   batch_size=256)


/tmp/ipykernel_1055/1669419674.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  model.fit(np.array([[df[textual_features].iloc[i],df[numeric_features].iloc[i],df[L_ohe].iloc[i] ]for i in range(10)]),y_train.iloc[:10],epochs=epochs,


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Series).

In [ ]:
ohe_features=["title","location","gender","year","country"]
textual_features=["company","level","tag","tag_truncated"]
numeric_features=["yearsofexperience","yearsatcompany"]

In [221]:
X_train

,company,level,tag,tag_truncated,yearsofexperience,yearsatcompany,title_Business Analyst,title_Data Scientist,title_Hardware Engineer,title_Human Resources,...,gender_Unavailable,year_2017,year_2018,year_2019,year_2020,year_2021,country_Canada,country_Germany,country_US,country_United Kingdom
21286,70,50,20,8,2.0,1.0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
19043,4,53,0,0,6.0,1.0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
17937,66,72,21,22,2.0,1.0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
18733,23,50,10,3,12.0,2.0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
22150,40,9,28,14,13.0,9.0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9016,39,41,39,22,3.0,0.0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
6478,26,63,22,9,11.0,0.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
16603,46,36,10,3,11.0,1.0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
10250,4,52,15,6,3.0,0.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [273]:
[X_train[textual_features],X_train[numeric_features],X_train[L_ohe]]

[       company  level  tag  tag_truncated
 21286       70     50   20              8
 19043        4     53    0              0
 17937       66     72   21             22
 18733       23     50   10              3
 22150       40      9   28             14
 ...        ...    ...  ...            ...
 9016        39     41   39             22
 6478        26     63   22              9
 16603       46     36   10              3
 10250        4     52   15              6
 26875        4     57   34             19
 
 [25208 rows x 4 columns],
        yearsofexperience  yearsatcompany
 21286                2.0             1.0
 19043                6.0             1.0
 17937                2.0             1.0
 18733               12.0             2.0
 22150               13.0             9.0
 ...                  ...             ...
 9016                 3.0             0.0
 6478                11.0             0.0
 16603               11.0             1.0
 10250                3.0          